In [14]:
import numpy as np
import pandas as pd


#### movies---> genre
movie_dict = {}
movie_map = {}

with open("./content/movies.dat", "r", encoding="latin1") as file:
    for line in file:
        # Split the line into MovieID, Title, and Genres
        parts = line.strip().split("::")
        movie_id = parts[0]
        title = parts[1]
        genres = parts[2].split("|")  # Split genres by "|"

        # Create a dictionary entry for the movie
        movie_dict[movie_id] = {
            'title': title,
            'genres': genres
        }
        movie_map[title] = {
            'id': movie_id
        }

Collaborative Filtering

In [15]:
rm = np.full((6041, 4000), np.nan)

In [16]:
with open("./content/ratings.dat", "r") as file:
    for line in file:
        # Split the line into UserID, MovieID, Rating, and Timestamp
        parts = line.strip().split("::")
        UserID = int(parts[0])
        MovieID = int(parts[1])
        Rating = float(parts[2])
        rm[UserID, MovieID] = Rating

In [17]:
rm = pd.DataFrame(rm)

In [18]:
rm.head()

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#item based collaborative filtering
nmpy = rm.values
nmpy = nmpy.T
print(nmpy)

[[nan nan nan ... nan nan nan]
 [nan  5. nan ... nan nan  3.]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [20]:
u = int(input("Enter Movie ID"))
list = []

num_rows, num_columns = nmpy.shape

mean = []
for v in range(num_rows):
    mean.append(np.nanmean(nmpy[v]))
# print(mean)


/tmp/ipykernel_60697/384280411.py:8: RuntimeWarning: Mean of empty slice
  mean.append(np.nanmean(nmpy[v]))


In [21]:
def new_pearson_correlation(X, Y, x, y):
    if(X.size == 0):
        return 0
    # Calculate the mean of X and Y
    mean_X = mean[x]
    mean_Y = mean[y]
    # Calculate the numerator
    numerator = np.sum((X - mean_X) * (Y - mean_Y))

    # Calculate the denominator
    denominator = np.sqrt(np.sum((X - mean_X) ** 2) * np.sum((Y - mean_Y) ** 2))
    if(denominator == 0):
        return 0
    # Calculate the Pearson correlation coefficient
    pearson_corr = numerator / denominator

    return pearson_corr


In [22]:
for column_index in range(num_columns):
    value = nmpy[u, column_index]
    if not np.isnan(value):
        list.append(column_index)
sim = []
for v in range(num_rows):
    ratings_u = []
    ratings_v = []
    for i in list:
        value = nmpy[v, i]
        if not np.isnan(value):
            ratings_u.append(nmpy[u, i])
            ratings_v.append(nmpy[v, i])
    ratings_u = np.array(ratings_u)
    ratings_v = np.array(ratings_v)
    sim.append(new_pearson_correlation(ratings_u, ratings_v, u, v))

In [23]:
movie = int(input("Enter user-id"))
num = 0
den = 0
for v in range(num_rows):
    if not np.isnan(nmpy[v, movie]):
        num += sim[v] * (nmpy[v, movie] - mean[v])
        den += abs(sim[v])
rating = (num/den) + mean[u]
print(rating)

2.8333836798248253
